In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Exchange Traded Market (European Options) Black-Scholes model

### Assumption/Parameters of Black-Scholes model:
* Current stock price: S0
* Strike price K
* Expiration (yr): T
* Risk-free rate (continuously compounded): r
* Volatility: σ

Calculate the prices of European put and call options. Demonstrate that put-call parity holds.

In [2]:
def blackscholes(S0, K, T, r, volatility, q=0):

    # Calculate values to use in Black-Scholes model
    t1 = np.log(S0/K) + (r - q + (volatility**2)/2)*T
    t2 = volatility*np.sqrt(T)
    d1 = t1/t2
    d2 = d1 - t2

    # Calculate price of European put options
    put_value = K*np.exp(-r*T)*norm(0,1).cdf(-d2) - S0*np.exp(-q*T)*norm(0,1).cdf(-d1)
    
    # Calculate price of European call options
    call_value = S0*np.exp(-q*T)*norm(0,1).cdf(d1) - K*np.exp(-r*T)*norm(0,1).cdf(d2)
    
    # Calculate put-call parity value
    pcp = K*np.exp(-r*T) + call_value - S0

    print(f'Put option value:\t\t${put_value:.2f}')
    print(f'Call option value:\t\t${call_value:.2f}')
    print(f'Put call parity:\t\t${pcp:.2f}')
    return

In [3]:
# Choose variables to use in black scholes equation
S0 = 72.25
K = 70
T = 1
r = 0.04
volatility = 0.35

blackscholes(S0, K, T, r, volatility)

Put option value:		$7.39
Call option value:		$12.39
Put call parity:		$7.39


> Viewing the output above, this demonstrates that put-call parity holds. We see that our put call parity value is equal to the previously calculated put option value, thus we know that any investment strategy involving a European call can also be done with a European put.

# Over-the-Counter Markets (Exotic Oprions) - Monte Carlo

In this case, option payoffs depend not only on the stock price at expiration but also upon the history of the stock price sampled at various points during the life of the option. Assume that the stock price will be needed at the end of each of the next ***12*** months, in addition to S0 today, and start by simulating ***10,000*** such stock price paths. In particular:

### Prep: 
* Generate random variables that follow a standard normal distribution. This will require a table with 10,000 rows and 12 columns. Denote the value of the random variable on path i for month j by Zi,j where i = 1, . . . ,10,000 and j = M1, . . . , M12.

* Generate a table with 10,000 stock price paths Si,j where i = 1, . . . ,10,000 and j = M0, . . . , M12. In each case, Si,0 = S0. For the remaining entries, define ∆t = 1/12 and set Si,j = Si,j−1 exp[(r − σ2/2)∆t + σ√∆tzi,j] for i = 1, . . . ,10,000 and j = 1, . . . ,12. We use risk-free rate r here instead of the expected rate of return on the underlying stock because of risk-neutral valuation. Note that ST on each simulated path i is defined to be equal to Si,12.

* Construct another table with 10,000 rows and 6 columns storing discounted payoffs (to the present values) at expiration for various different options, as indicated below:
    * European call option, with payoff max(ST − K,0).
    * European put option, with payoff max(K − ST ,0).
    * Average price put option, with payoff max(K −  ^S,0), where ^S = (S0 + S1 + · · · + S12) / 13.
    * Floating lookback call option, with payoff ST − Smin, where Smin on each path is the minimum of the stock prices S0, S1, . . . , S12 on that path.
    * Up-and-out call option with barrier B = $90, with payoff max(ST −K,0) as long as none of the observed monthly stock prices on the given path are greater than or equal to B. If any of the monthly stock prices are greater than or equal to B, then the payoff is zero.
    * Up-and-in call option with barrier B = $90, with payoff max(ST − K,0) as long as at least one of the observed monthly stock prices on the given path is greater than or equal to B. If this condition is not satisfied, the option payoff is zero.

* Create another table which shows the following quantities for each of the options listed above (below N is the number of simulated paths, which is 10,000 here):
    * Mean discounted payoff μ. This is the estimated option premium.
    * Standard error of option premium φ. This is the standard deviation of the option payoff divided by √N .
    * Lower bound for 95% confidence interval for option premium, which is μ − 1.96φ.
    * Upper bound for 95% confidence interval for option premium, which is μ + 1.96φ.

In [4]:
# Choose seed value and generate random variables that follow a standard normal distribution
SEED = 0 # 2308974
rng = np.random.default_rng(SEED)
z = rng.normal(0, 1, size =(100000,12))

rand_var_normal = pd.DataFrame(z)
rand_var_normal.index += 1
rand_var_normal.columns = ["M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "M10", "M11", "M12"]
rand_var_normal

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
1,0.125730,-0.132105,0.640423,0.104900,-0.535669,0.361595,1.304000,0.947081,-0.703735,-1.265421,-0.623274,0.041326
2,-2.325031,-0.218792,-1.245911,-0.732267,-0.544259,-0.316300,0.411631,1.042513,-0.128535,1.366463,-0.665195,0.351510
3,0.903470,0.094012,-0.743499,-0.921725,-0.457726,0.220195,-1.009618,-0.209176,-0.159225,0.540846,0.214659,0.355373
4,-0.653829,-0.129614,0.783975,1.493431,-1.259066,1.513924,1.345875,0.781311,0.264456,-0.313923,1.458021,1.960258
5,1.801635,1.315104,0.357380,-1.208319,-0.004454,0.656475,-1.288361,0.395122,0.429864,0.696043,-1.184118,-0.661703
...,...,...,...,...,...,...,...,...,...,...,...,...
99996,-0.670701,-0.767568,-0.065244,1.984216,1.318853,1.444852,1.106874,0.562651,1.427445,0.938100,-0.467421,-2.043927
99997,-1.214928,-0.885780,-0.152534,-1.955656,-0.212429,0.739953,0.724015,-0.178199,-0.170225,-0.936635,-0.848505,-0.001502
99998,0.235193,0.788047,-0.584540,-1.474101,-1.026918,0.213084,3.216678,0.779026,-1.646539,-0.778026,0.975835,0.060701
99999,1.299671,0.851758,0.006445,-0.715954,0.282738,1.362345,-1.252084,-1.345153,0.376343,0.099493,0.219761,0.318852


In [5]:
# Option values
S0 = 72.25
K = 70
T = 1
r = 0.04
sigma = 0.35

price_paths = pd.DataFrame(index=range(100000), columns=range(13))
price_paths.index += 1 # Reindex dataframe to start from 1 
price_paths.columns = ["M0", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "M10", "M11", "M12"]

# Generating stock price paths
price_paths.iloc[:,0] = S0 # Si,0 = S0
t = 1/12 # the rest of the columns are calculated using the previous column
for j in range(1, 13):
    price_paths.iloc[:, j] = price_paths.iloc[:, j - 1] 
    price_paths.iloc[:, j] *= np.exp((r-sigma**2/2)*t + sigma*np.sqrt(t)*rand_var_normal.iloc[:, j-1])

price_paths

,M0,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
1,72.25,73.044205,71.948213,76.621804,77.301207,73.099147,75.685007,86.190554,94.677777,88.023663,77.322218,72.474673,72.649155
2,72.25,57.022675,55.677285,49.004846,45.42955,42.922752,41.499169,43.184956,47.897009,47.195359,54.086636,50.481528,52.214067
3,72.25,79.015558,79.628542,73.735278,67.059669,63.915701,65.237986,58.807123,57.475446,56.458141,59.523649,60.721099,62.829577
4,72.25,67.511711,66.51547,71.871062,83.428804,73.333056,85.30234,97.554708,105.3811,108.043301,104.485014,120.854317,147.064643
5,72.25,86.52142,98.641729,102.08767,90.195278,89.995189,95.996639,84.130661,87.402327,91.12051,97.586353,86.429403,80.696977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,72.25,67.396717,62.257139,61.738662,75.310332,85.892672,99.217077,110.760891,117.032426,134.94995,148.104083,141.022285,114.507036
99997,72.25,63.790842,58.226635,57.234713,46.889673,45.812801,49.281799,52.928172,51.891778,50.916681,46.237246,42.363412,42.282043
99998,72.25,73.856539,79.83604,75.124324,64.614309,58.143206,59.303445,81.932666,88.485337,74.791618,69.015158,76.031679,76.364067
99999,72.25,82.242762,89.475341,89.375215,82.991253,85.245145,97.651656,85.895366,74.847281,77.610544,78.255952,79.871406,82.34047


In [6]:
discounted_payoffs = pd.DataFrame(index=range(100000), columns=range(6))
discounted_payoffs.index +=1 # Reindex dataframe to start from 1 
discounted_payoffs.columns = ["European Call", "European Put", "Average Price Put", "Floating Lookback Call", "Up-and-out Call", "Up-and-in Call"]

discount_factor = np.exp(-r * T)
B = 90

# Column 1 stores discounted payoffs at expiration for European call options
discounted_payoffs.iloc[:, 0] = ((price_paths.iloc[:, 12]-K) * discount_factor).clip(lower=0)

# Column 2 stores discounted payoffs at expiration for European put options
discounted_payoffs.iloc[:, 1] = (K-(price_paths.iloc[:, 12]) * discount_factor).clip(lower=0)

# Column 3 stores discounted payoffs at expiration for average price put option
average_price = price_paths.sum(axis=1) / 13
discounted_payoffs.iloc[:, 2] = ((K-average_price) * discount_factor).clip(lower=0)

# Column 4 stores discounted payoffs at expiration for floating lookback call options
s_min = price_paths.min(axis=1)
discounted_payoffs.iloc[:, 3] = (price_paths.iloc[:, 12]-s_min) * discount_factor

# Column 5 stores discounted payoffs at expiration for up-and-out call options
s_max = price_paths.max(axis=1)
discounted_payoffs.iloc[:, 4] = np.where(s_max>=B, 0, ((price_paths.iloc[:, 12]-K) * discount_factor).clip(lower=0))

# Column 6 stores discounted payoffs at expiration for up-and-in call options
discounted_payoffs.iloc[:, 5] = np.where(s_max>=B, ((price_paths.iloc[:, 12]-K) * discount_factor).clip(lower=0), 0)

discounted_payoffs

,European Call,European Put,Average Price Put,Floating Lookback Call,Up-and-out Call,Up-and-in Call
1,2.54528,0.19946,0,0.673458,0,2.54528
2,0,19.833276,18.560543,10.29476,0,0
3,0,9.634006,3.942358,6.121609,0,0
4,74.042895,0,0,77.390795,0,74.042895
5,10.277543,0,0,8.115767,0,10.277543
...,...,...,...,...,...,...
99996,42.76189,0,0,50.699296,0,42.76189
99997,0,29.37586,16.990764,0.0,0,0
99998,6.114528,0,0,17.50641,6.114528,0
99999,11.856593,0,0,9.694817,0,11.856593


In [7]:
N = 100000
stats = pd.DataFrame(index=range(4), columns=range(6))

# Row 1 contains the mean discounted payoff
stats.iloc[0,:] = discounted_payoffs.iloc[:,:].mean()

# Row 2 is the standard error of the discounted payoff
stats.iloc[1,:] = discounted_payoffs.iloc[:,:].std() / np.sqrt(N)

# Row 3 is the lower bound for 95% confidence interval of the discounted payoff
stats.iloc[2,:] = stats.iloc[0,:] - 1.96*stats.iloc[1,:]

# Row 4 is the upper bound for 95% confidence interval of the discounted payoff
stats.iloc[3,:] = stats.iloc[0,:] + 1.96*stats.iloc[1,:]

stats.columns = ["European Call", "European Put", "Average Price Put", "Floating Lookback Call", "Up-and-out Call", "Up-and-in Call"]
stats.index = ["Mean (μ)", "Standard Error (φ)", "Lower Bound (μ-1.96φ)", "Upper Bound (μ+1.96φ)"]
stats = stats.astype(float).round(decimals=4)
stats

,European Call,European Put,Average Price Put,Floating Lookback Call,Up-and-out Call,Up-and-in Call
Mean (μ),12.4030,8.7542,3.8206,16.3154,1.1070,11.2960
Standard Error (φ),0.0625,0.0355,0.0184,0.0605,0.0105,0.0636
Lower Bound (μ-1.96φ),12.2805,8.6846,3.7845,16.1967,1.0864,11.1713
Upper Bound (μ+1.96φ),12.5255,8.8238,3.8568,16.4340,1.1276,11.4207


### Improvement:
> One disadvantage of Monte Carlo methods is that they can be slow to converge. On the other hand, there are situations where they are the only methods that can be used. As a result, ***antithetic variates*** is introduced there reduce the vairance and improve the rate of convergence.  This relies on the fact that if Z is a standard normal random variable, then so is −Z.

* Generate another 10,000 stock price paths, but this time using −Zi,j instead of Zi,j throughout.

* Provide another table of variates discounted payoffs, report the average discounted payoff along each path i from the cases which have Zi,j and the cases which have −Zi,j. 

* Make a final table similar to the last one from part *Prep*, containing the mean average discounted payoff, its standard error, and the 95% confidence interval bounds. Note that when calculating the standard error, continue to use **N = 10,000**. This is because we still have only 10,000 independent paths, even though there are 20,000 paths in total.

In [8]:
variates_price_paths = pd.DataFrame(index=range(100000), columns=range(13))
variates_price_paths.index += 1 # Reindex dataframe to start from 1
variates_price_paths.columns = ["M0", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "M10", "M11", "M12"]
t = 1/12

# Calculate stock price paths using -Zi,j
variates_price_paths.iloc[:,0] = S0
for j in range(1, 13):
    variates_price_paths.iloc[:, j] = variates_price_paths.iloc[:, j - 1] 
    variates_price_paths.iloc[:, j] *= np.exp((r-sigma**2/2)*t + sigma*np.sqrt(t)*(-1*rand_var_normal.iloc[:, j-1]))

variates_price_paths

,M0,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12
1,72.25,71.211775,72.040952,67.407613,66.578947,70.157283,67.520724,59.081178,53.594798,57.442473,65.16136,69.273966,68.863267
2,72.25,91.219984,93.093937,105.395554,113.288221,119.480632,123.14238,117.916976,105.940569,107.135469,93.154635,99.454358,95.814374
3,72.25,65.830168,65.09246,70.04643,76.747068,80.237523,78.333296,86.592222,88.285288,89.558331,84.645698,82.683089,79.625844
4,72.25,77.047485,77.924995,71.863318,61.688922,69.933504,59.908163,52.198808,48.151294,46.7988,48.221469,41.542645,34.018089
5,72.25,60.119303,52.545892,50.592721,57.061002,56.985686,53.234223,60.527749,58.056078,55.490217,51.630384,58.08912,61.995607
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,72.25,77.178945,83.254993,83.657351,68.339003,59.70751,51.50632,45.975068,43.357525,37.467942,34.01946,35.601522,43.6904
99997,72.25,81.541603,89.017984,90.240568,109.760478,111.94333,103.695616,96.210379,97.784978,99.305312,108.968879,118.512834,118.321108
99998,72.25,70.42853,64.923282,68.751273,79.651598,88.203557,86.172168,62.151517,57.345505,67.605127,73.004554,66.033107,65.513249
99999,72.25,63.246994,57.929008,57.788873,62.014161,60.161052,52.332,59.284216,67.794531,65.149613,64.383867,62.858641,60.758193


In [9]:
# Create a dataframe to calculate the discounted payoffs at expiration using the antithetic variate price paths
variate_payoffs = pd.DataFrame(index=range(100000), columns=range(6))
variate_payoffs.index += 1 # Reindex dataframe to start at 1
variate_payoffs.columns = ["European Call", "European Put", "Average Price Put", "Floating Lookback Call", "Up-and-out Call", "Up-and-in Call"]

discount_factor = np.exp(-r * T)
B = 90

# Column 1 stores discounted payoffs at expiration for European call options
variate_payoffs.iloc[:, 0] = ((variates_price_paths.iloc[:, 12]-K) * discount_factor).clip(lower=0)

# Column 2 stores discounted payoffs at expiration for European put options
variate_payoffs.iloc[:, 1] = (K-(variates_price_paths.iloc[:, 12]) * discount_factor).clip(lower=0)

# Column 3 stores discounted payoffs at expiration for average price put option
average_price = variates_price_paths.sum(axis=1) / 13
variate_payoffs.iloc[:, 2] = ((K-average_price) * discount_factor).clip(lower=0)

# Column 4 stores discounted payoffs at expiration for floating lookback call options
s_min = variates_price_paths.min(axis=1)
variate_payoffs.iloc[:, 3] = (variates_price_paths.iloc[:, 12]-s_min) * discount_factor

# Column 5 stores discounted payoffs at expiration for up-and-out call options
s_max = variates_price_paths.max(axis=1)
variate_payoffs.iloc[:, 4] = np.where(s_max>=B, 0, ((variates_price_paths.iloc[:, 12]-K) * discount_factor).clip(lower=0))

# Column 6 stores discounted payoffs at expiration for up-and-in call options
variate_payoffs.iloc[:, 5] = np.where(s_max>=B, ((variates_price_paths.iloc[:, 12]-K) * discount_factor).clip(lower=0), 0)

variate_payoffs

,European Call,European Put,Average Price Put,Floating Lookback Call,Up-and-out Call,Up-and-in Call
1,0,3.8369,3.652158,14.669784,0,0
2,24.802178,0,0,22.640402,0,24.802178
3,9.248409,0,0,13.963522,9.248409,0
4,0,37.315779,10.971662,0.0,0,0
5,0,10.435275,11.930198,10.955773,0,0
...,...,...,...,...,...,...
99996,0,28.022726,12.859351,9.291737,0,0
99997,46.42641,0,0,44.264634,0,46.42641
99998,0,7.055562,0,7.847482,0,0
99999,0,11.62417,7.689926,8.095797,0,0


In [10]:
avg_payoffs = pd.DataFrame(index=range(100000), columns=range(6))
avg_payoffs.index += 1 # Reindex dataframe to start at 1
avg_payoffs.columns = ["European Call", "European Put", "Average Price Put", "Floating Lookback Call", "Up-and-out Call", "Up-and-in Call"]

# Calculate the averages discounted payoff for each path's options
# Note that the discounted_payoffs stores the discounted payoffs on the paths with Zij, while variate_payoffs stores the discounted payoffs on the paths with -Zij
for idx in range(6):
    avg_payoffs.iloc[:,idx] = (discounted_payoffs.iloc[:,idx] + variate_payoffs.iloc[:,idx]) / 2

avg_payoffs

,European Call,European Put,Average Price Put,Floating Lookback Call,Up-and-out Call,Up-and-in Call
1,1.27264,2.01818,1.826079,7.671621,0.0,1.27264
2,12.401089,9.916638,9.280271,16.467581,0.0,12.401089
3,4.624205,4.817003,1.971179,10.042565,4.624205,0.0
4,37.021448,18.65789,5.485831,38.695397,0.0,37.021448
5,5.138771,5.217638,5.965099,9.53577,0.0,5.138771
...,...,...,...,...,...,...
99996,21.380945,14.011363,6.429675,29.995516,0.0,21.380945
99997,23.213205,14.68793,8.495382,22.132317,0.0,23.213205
99998,3.057264,3.527781,0.0,12.676946,3.057264,0.0
99999,5.928297,5.812085,3.844963,8.895307,0.0,5.928297


In [11]:
N = 100000
avg__stats = pd.DataFrame(index=range(4), columns=range(6))

# Row 1 contains the mean average discounted payoff
avg__stats.iloc[0,:] = avg_payoffs.iloc[:,:].mean()

# Row 2 is the standard error of the average discounted payoffs
avg__stats.iloc[1,:] = avg_payoffs.iloc[:,:].std() / np.sqrt(N)

# Row 3 is the lower bound for 95% confidence interval for option premium
avg__stats.iloc[2,:] = avg__stats.iloc[0,:] - 1.96*avg__stats.iloc[1,:]

# Row 4 is the upper bound for 95% confidence interval for option premium
avg__stats.iloc[3,:] = avg__stats.iloc[0,:] + 1.96*avg__stats.iloc[1,:]

avg__stats.columns = ["European Call", "European Put", "Average Price Put", "Floating Lookback Call", "Up-and-out Call", "Up-and-in Call"]
avg__stats.index = ["Mean (μ)", "Standard Error (φ)", "Lower Bound (μ-1.96φ)", "Upper Bound (μ+1.96φ)"]
avg__stats = avg__stats.astype(float).round(decimals=4)
avg__stats

,European Call,European Put,Average Price Put,Floating Lookback Call,Up-and-out Call,Up-and-in Call
Mean (μ),12.3528,8.7828,3.8348,16.2684,1.0999,11.2529
Standard Error (φ),0.0343,0.0158,0.0099,0.0290,0.0070,0.0371
Lower Bound (μ-1.96φ),12.2855,8.7519,3.8154,16.2114,1.0862,11.1801
Upper Bound (μ+1.96φ),12.4201,8.8136,3.8541,16.3253,1.1135,11.3256


## Thinking:
Among the four exotic options above (i.e. the average price put option, the floating lookback call option, the up-and-out call option, the up-and-in call option), the ***floating lookback call option*** might be useful to a corporation when the corporation is very confident that the value of an option will rise. This option could be purchased instead of a standard European call option.

The payoff of this option would be preferred to that of a standard call option because its mean discounted payoff is more than a European call option's. Additionally, it has a smaller standard error than the European call option. Both of these values are true for regular and antithetic variate calculations. Hence its estimates are more accurate and the discounted payoff is more likely to be closer to the mean.

Although calls amplify risk and expected return, floating lookback calls have an even larger mean and smaller standard error than standard European calls.